#### Библиотеки

In [2]:
import torch
import torchvision
import matplotlib.pyplot as plt
from pathlib import Path
from torchvision.transforms import ToTensor
from torchvision import transforms
from torch.utils.data import DataLoader
import os
from torch import nn
from tqdm.auto import tqdm
from torchvision import models
from torchinfo import summary

#### Функции

In [4]:
def make_data(train_dir_path, test_dir_path, trans, batch_size, num_workers):

    train_data = torchvision.datasets.ImageFolder(root=train_dir_path, transform=trans)
    test_data = torchvision.datasets.ImageFolder(root=test_dir_path, transform=trans)

    train_dataloader = DataLoader(dataset=train_data,
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True,
    num_workers=num_workers)
    test_dataloader = DataLoader(dataset=test_data,
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True,
    num_workers=num_workers)
    classes_names = train_data.classes

    return train_dataloader, test_dataloader, classes_names

In [5]:
from torch.cuda import is_available
device = "cuda" if is_available() else "cpu"
device

'cuda'

In [6]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct/len(y_pred)) * 100
    return acc

In [7]:
def train_model(model, train_dataloader, test_dataloader, loss_fn, optimizer, accuracy_fn, epochs, device):
    train_epoch_count = []
    train_loss_values = []
    train_acc_values = []
    test_epoch_count = []
    test_loss_values = []
    test_acc_values = []
    for epoch in tqdm(range(epochs)):
        # Train Block
        ls_loss = 0
        ls_acc = 0
        for batch, (X, y) in enumerate(train_dataloader):
            X, y = X.to(device), y.to(device)
            #y = y.type(torch.long)
            model.train()
            logits = model(X).squeeze()
            preds = torch.round(torch.sigmoid(logits))
            #print(X.shape, y.shape, logits.shape)
            loss = loss_fn(logits, y.float())
            acc = accuracy_fn(y_true = y, y_pred = preds)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            model.eval()
            ls_loss += loss.item()
            ls_acc += acc
            if batch % 400 == 0:
                print(f"Epoch: {epoch+1} | Batch: {batch}...")
        mean_loss = ls_loss / len(train_dataloader)
        mean_acc = ls_acc / len(train_dataloader)
        train_loss_values.append(loss.item())
        train_acc_values.append(acc)
        train_epoch_count.append(epoch+1)
        print(f"Epoch: {epoch+1} | Loss: {mean_loss:.4f} | Acc: {mean_acc:.2f}%")


        # Test Block 
        ls_loss, ls_acc = 0, 0
        for batch, (X, y) in enumerate(test_dataloader):
            with torch.inference_mode():
                model.eval()
                X, y = X.to(device), y.to(device)
                #y = y.type(torch.long)
                mean_loss, mean_acc = 0, 0
                logits = model(X).squeeze()
                preds = torch.round(torch.sigmoid(logits))
                loss = loss_fn(logits, y.float())
                acc = accuracy_fn(y_true = y,y_pred = preds)
                ls_loss += loss.item()
                ls_acc += acc
                if batch % 300 == 0:
                    print(f"TEST :    Epoch: {epoch+1} | Batch: {batch}...")
        mean_loss = ls_loss / len(test_dataloader)
        mean_acc = ls_acc / len(test_dataloader)
        test_loss_values.append(loss.item())
        test_acc_values.append(acc)
        test_epoch_count.append(epoch+1)
        print(f"TEST :    Epoch: {epoch+1} | Loss: {mean_loss:.4f} | Acc: {mean_acc:.2f}%\n\n")

    return train_loss_values, train_acc_values, test_loss_values, test_acc_values, train_epoch_count, test_epoch_count

In [8]:
def plots(train_loss_values, train_acc_values, test_loss_values, test_acc_values, train_epoch_count, test_epoch_count):
    plt.subplot(2, 1, 1)
    plt.plot(train_epoch_count, train_acc_values, c='g', label='Train Acc')
    plt.plot(train_epoch_count, test_acc_values, c='r', label='Test ACC')
    plt.title('Acc')
    plt.legend()
    plt.subplot(2, 1, 2)
    plt.plot(train_epoch_count, train_loss_values, c='blue', label="Train Loss")
    plt.plot(train_epoch_count, test_loss_values, c='orange', label="Test Loss")
    plt.title('Loss')
    plt.legend();

#### Пути файлов и функция трансоформации

In [3]:
train_dir_path = Path("E:/CODING/projects/data/food_data/train")
test_dir_path =  Path("E:/CODING/projects/data/food_data/test")

trans = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])
cpu_c = os.cpu_count()
    

#### Модель EfficientNet_B0

##### Скачиваю веса модели, и модель

In [9]:
from torchvision import models
weights = models.EfficientNet_B0_Weights.DEFAULT

In [10]:
auto_transform = weights.transforms()

In [11]:
auto_transform

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [71]:
AlexV0_transfer_efficientnet_b0 = models.efficientnet_b0(weights=weights).to(device)

##### Получаю даталоадеры и классы

In [72]:
train_dataloader, test_dataloader, classes_names = make_data(train_dir_path=train_dir_path, test_dir_path=test_dir_path, trans=auto_transform, 
                                                            batch_size=32, num_workers=cpu_c)

In [73]:
AlexV0_transfer_efficientnet_b0

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [74]:
from torchinfo import summary
summary(model=AlexV0_transfer_efficientnet_b0,
        input_size=(32, 3, 224, 224),
        col_names=['input_size', 'output_size',
                   "num_params", 'trainable'],
                   col_width=20,
                   row_settings=['var_names'])


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   64                   True
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 16, 112

##### Отключаю обучение у всех слоев

In [26]:
for params in AlexV0_transfer_efficientnet_b0.features.parameters():
   params.requires_grad=False
from torchinfo import summary
summary(model=AlexV0_transfer_efficientnet_b0,
        input_size=(32, 3, 224, 224),
        col_names=['input_size', 'output_size',
                   "num_params", 'trainable'],
                   col_width=20,
                   row_settings=['var_names'])


Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

##### Переписываю последний слой, чтобы получать бинарный ответ

In [75]:
AlexV0_transfer_efficientnet_b0.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1280, out_features=1, bias=True)
).to(device)

##### Обучение модели

In [ ]:
optimizerV0 = torch.optim.SGD(AlexV0_transfer_efficientnet_b0.parameters(),
                              lr=0.01)
loss_fn = nn.BCEWithLogitsLoss()
train_loss_valuesV0, train_acc_valuesV0, test_loss_valuesV0, test_acc_valuesV0, train_epoch_countV0, test_epoch_countV0 = train_model(
    model=AlexV0_transfer_efficientnet_b0, train_dataloader=train_dataloader, test_dataloader=test_dataloader, 
            optimizer=optimizerV0, loss_fn=loss_fn ,accuracy_fn=accuracy_fn, epochs=10, device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | Batch: 0...
Epoch: 1 | Batch: 400...
Epoch: 1 | Batch: 800...
Epoch: 1 | Batch: 1200...
Epoch: 1 | Loss: 0.2871 | Acc: 88.34%
TEST :    Epoch: 1 | Batch: 0...
TEST :    Epoch: 1 | Batch: 300...


 10%|█         | 1/10 [06:05<54:50, 365.60s/it]

TEST :    Epoch: 1 | Loss: 0.2085 | Acc: 92.35%


Epoch: 2 | Batch: 0...
Epoch: 2 | Batch: 400...
Epoch: 2 | Batch: 800...
Epoch: 2 | Batch: 1200...
Epoch: 2 | Loss: 0.1859 | Acc: 92.80%
TEST :    Epoch: 2 | Batch: 0...
TEST :    Epoch: 2 | Batch: 300...


 20%|██        | 2/10 [11:44<46:40, 350.03s/it]

...

TEST :    Epoch: 8 | Loss: 0.1812 | Acc: 93.76%


Epoch: 9 | Batch: 0...
Epoch: 9 | Batch: 400...
Epoch: 9 | Batch: 800...
Epoch: 9 | Batch: 1200...
Epoch: 9 | Loss: 0.0845 | Acc: 96.80%
TEST :    Epoch: 9 | Batch: 0...
TEST :    Epoch: 9 | Batch: 300...


 90%|█████████ | 9/10 [52:44<05:57, 357.76s/it]

TEST :    Epoch: 9 | Loss: 0.2143 | Acc: 93.05%


Epoch: 10 | Batch: 0...
Epoch: 10 | Batch: 400...
Epoch: 10 | Batch: 800...
Epoch: 10 | Batch: 1200...
Epoch: 10 | Loss: 0.0776 | Acc: 97.06%
TEST :    Epoch: 10 | Batch: 0...
TEST :    Epoch: 10 | Batch: 300...


100%|██████████| 10/10 [58:46<00:00, 352.60s/it]

TEST :    Epoch: 10 | Loss: 0.1982 | Acc: 94.58%




##### Сохраняю модель

In [9]:
def save_model(model, MODEL_PATH, model_name):
    SAVE_PATH = MODEL_PATH / model_name
    torch.save(obj = model.state_dict(),
               f = SAVE_PATH)
    print(f"Saved to: {SAVE_PATH}")

In [ ]:
MODEL_PATH_AlexV0_transfer_efficientnet_b0 = Path("E:/CODING/projects/transfer_learning_models")
MODEL_NAME_AlexV0_transfer_efficientnet_b0 = "first_model_effnet_b0.pth"
MODEL_PATH_AlexV0_transfer_efficientnet_b0 / MODEL_NAME_AlexV0_transfer_efficientnet_b0
save_model(AlexV0_transfer_efficientnet_b0, MODEL_PATH_AlexV0_transfer_efficientnet_b0, MODEL_NAME_AlexV0_transfer_efficientnet_b0)

Saved to: E:\CODING\projects\transfer_learning_models\AlexV0_transfer_efficientnet_b0.pth
